# Computational study involving potential energy calculation of saturated alkanes

In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


## Code for Computation

In [ ]:

import numpy as np
INPUT_FILE = "inputMol" 

# C++ brain so I like to keep my constants separate
class molParam:
    kb = {"C-C": 300.0, "C-H": 350.0} 
    r0 = {"C-C": 1.53, "C-H": 1.11} 
    ka = {"C-C-C": 60.0, "H-C-H": 35.0, "C-C-H": 35.0} 
    theta0 = {"C-C-C": 109.5, "H-C-H": 109.5, "C-C-H": 109.5} 
    aPhi = {"C-C-C-C": 0.3, "H-C-C-H": 0.3, "C-C-C-H": 0.3}
    vdW_epsilon = {"C": 0.07, "H": 0.03} 
    vdW_sigma = {"C": 1.75, "H": 1.2} 

atomMapping = ["C", "H"] # 0 for Carbon, 1 for Hydrogen

# Binary symbols ko map karne ke liye
def atomType(binary):
    return atomMapping[binary] 

# These two subroutine are self explanatory
def calcDist(a1, a2):
    dist = np.linalg.norm(a1[:3] - a2[:3]) 
    return dist

def calcAngle(a1, a2, a3):
    vec1 = a1[:3] - a2[:3]
    vec2 = a3[:3] - a2[:3]

    cos_theta = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    angle = np.arccos(cos_theta) * (180.0 / np.pi)

    return angle

# This is a very messy way (I admit) to construct the 
# chain that we are working with. But oh well, it works.
def constbondType(*atoms):
    n = len(atoms)
    types = [atomMapping[int(a[3])] for a in atoms]

    if n == 2:

        t1, t2 = types
        return t1 + "-" + t2 if t1 <= t2 else t2 + "-" + t1
    
    elif n == 3:

        t1, t2, t3 = types
        return t1 + "-" + t2 + "-" + t3 if t1 <= t3 else t3 + "-" + t2 + "-" + t1
    
    elif n == 4:

        t1, t2, t3, t4 = types
        return t1 + "-" + t2 + "-" + t3 + "-" + t4 if t1 <= t4 else t4 + "-" + t2 + "-" + t3 + "-" + t1
    

#----------------------------------# 
# Subroutines to calculate energies
#----------------------------------#

def calcBondStrtchEnergy(a1, a2):
    distance = calcDist(a1, a2)
    bondType = constbondType(a1, a2)

    stretchEnergy = molParam.kb[bondType] * (distance - molParam.r0[bondType])**2

    return stretchEnergy

# a2 is always a carbon atom
def calcAngleBendEnergy(a1, a2, a3):
    angleType = constbondType(a1, a2, a3)

    angle = calcAngle(a1, a2, a3)
    bendEnergy = molParam.ka[angleType] * ((angle - molParam.theta0[angleType])*(np.pi/180.0))**2

    return bendEnergy

def calcVndrwlEnergy(a1, a2):
    distance = float(calcDist(a1, a2))
    
    epsilon = np.sqrt(float(molParam.vdW_epsilon[atomType(int(a1[3]))]) * float(molParam.vdW_epsilon[atomType(int(a2[3]))]))
    sigma = 2 * np.sqrt(float(molParam.vdW_sigma[atomType(int(a1[3]))]) * float(molParam.vdW_sigma[atomType(int(a2[3]))])) 
    
    vdwEnergy = 4 * epsilon * ((sigma / distance)**12 - (sigma / distance)**6)

    return vdwEnergy

def calcDihlAngle(a1, a2, a3, a4):
    dihedralType = constbondType(a1, a2, a3, a4)
    
    r12 = a1[:3] - a2[:3]
    r32 = a3[:3] - a2[:3]
    r23 = a2[:3] - a3[:3]
    r43 = a4[:3] - a3[:3]

    t = np.cross(r12, r32)
    u = np.cross(r23, r43)
    cos_phi = np.dot(t, u) / (np.linalg.norm(t) * np.linalg.norm(u))
    phi = np.arccos(cos_phi)

    return phi

# a2 and a3 are always carbon atoms
def calcDihlEnergy(a1, a2, a3, a4):
    dihedralType = constbondType(a1, a2, a3, a4)

    phi = calcDihlAngle(a1,a2,a3,a4)
    
    dihedralEnergy = molParam.aPhi[dihedralType] * (1 + np.cos(3 * phi))
    return dihedralEnergy

#----------------------------------#
# Main stuff starts here
#----------------------------------#

f = open(INPUT_FILE, 'r')
lines = f.readlines()

numAtomInfo = lines[0].split()

# Basic initialization of the alkane in terms of number of atoms and bonds
nat = int(numAtomInfo[0])
nbonds = int(numAtomInfo[1])
numCAtoms = int(numAtomInfo[2])
numCCBond = int(numAtomInfo[3])

rAtoms = np.zeros((nat, 4)) # Array that stores the coordinates from your prof's file

for i in range(1, nat+1): 


    lineNo = i
    coordAtomInfo = lines[lineNo].split()

    currentAtom = i-1

    for k in range(0, 3):

        rAtoms[currentAtom][k] = float(coordAtomInfo[k])
    
    rAtoms[currentAtom][3] = 0 if (coordAtomInfo[3] == "C") else 1 # 0 for Carbon, 1 for Hydrogen


bondList = [] # List to store bond info


for i in range ( (nat+1), (nat+1) + nbonds):

    bondInfo = lines[i].split()

    bondList.append((int(bondInfo[0]) - 1, int(bondInfo[1]) - 1))


# Connectivity list for the atoms

f.close()

bondConnectivity = {}

for a,b in bondList:

    if a not in bondConnectivity:
        bondConnectivity[a] = []
    if b not in bondConnectivity:
        bondConnectivity[b] = []
    
    bondConnectivity[a].append(b)
    bondConnectivity[b].append(a)

# The reason for creating this is that it allows me to easily
# loop through the bonds

# Now let's calculate the individual sets
angleTriplets = []
dihedralQuartets = []
vanderWaalsPairs = []

for i in bondConnectivity:
    
    # Angle Triplets
    for j in bondConnectivity[i]:
        for k in bondConnectivity[i]:
           if (j>k): angleTriplets.append((j,i,k))
    
    # Dihedral Quartets
    for j in bondConnectivity[i]:
        if (rAtoms[i][3] == 0 and rAtoms[j][3] == 0): # Both Carbon atoms
            if (i>j):
                for k in bondConnectivity[j]:
                    if (k != i):
                        for l in bondConnectivity[i]:
                            if (l != j):
                                dihedralQuartets.append((l, i, j, k))


    # VanderWaals Pairs
    for j in range(i):
        if j in bondConnectivity[i]:
            continue
        
        common_neighbor = False
        for k in bondConnectivity[j]:
            if k in bondConnectivity[i]:
                common_neighbor = True
                break
        if common_neighbor:
            continue

        # valid vdW pair
        vanderWaalsPairs.append((i, j))


# Setting (hahaha) is done, now to loop through and calculate stuff

#----------------------------------#
# Final calculations (finally)
#----------------------------------#
stretchEnergyTotal, bendEnergyTotal, dihlEnergyTotal, vdwEnergyTotal = 0.0, 0.0, 0.0, 0.0

for a,b in bondList:
    stretchEnergyTotal += calcBondStrtchEnergy(rAtoms[a], rAtoms[b])

for a,b,c in angleTriplets:
    bendEnergyTotal += calcAngleBendEnergy(rAtoms[a], rAtoms[b], rAtoms[c])

for a,b,c,d in dihedralQuartets:
    dihlEnergyTotal += calcDihlEnergy(rAtoms[a], rAtoms[b], rAtoms[c], rAtoms[d])

for a,b in vanderWaalsPairs:
    vdwEnergyTotal += calcVndrwlEnergy(rAtoms[a], rAtoms[b])

# Simpal, now printing
print("Bond Stretching Energy: ", stretchEnergyTotal)
print("Angle Bending Energy: ", bendEnergyTotal)
print("Dihedral Energy: ", dihlEnergyTotal)
print("VanderWaals Energy: ", vdwEnergyTotal)


Bond Stretching Energy:  6.257863558476223
Angle Bending Energy:  18.92702787133004
Dihedral Energy:  17.4220294165852
VanderWaals Energy:  26.60706420487743
